In [1]:
import torch

# torch.get_default_device() 함수가 없다고 떠서 대체 함수 정의


def _get_default_device():
    """torch.get_default_device() 대체 함수"""

    # gpu가 있으면 cuda 디바이스로 반환
    if torch.cuda.is_available():
        return torch.device("cuda")
    # apple m1,m2 칩이 있으면 mps 디바이스로 반환
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    # 그 외는 cpu 디바이스로 반환
    else:
        return torch.device("cpu")

## 라이브러리 임포트

In [2]:
import os
import torch
import pickle
import copy
from dotenv import load_dotenv
from langchain_core.prompts import load_prompt
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_teddynote import logging
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
import torch.nn.functional as F
import re
from soynlp.normalizer import repeat_normalize
import pandas as pd
import os
import pickle
import copy
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import torch.nn.functional as F
import re
from soynlp.normalizer import repeat_normalize
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from langchain_core.prompts import load_prompt

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# 프로젝트 이름을 입력합니다.
logging.langsmith("chatnge_ai")

llm = ChatOpenAI()

LangSmith 추적을 시작합니다.
[프로젝트명]
chatnge_ai


In [3]:
load_dotenv()


True

In [4]:
# 클로드 api 키를 환경 변수에서 불러옴
api_key2 = os.getenv("ANTHROPIC_API_KEY")

# 제미나이 api 키를 환경 변수에서 불러옴
api_key3 = os.getenv("GOOGLE_API_KEY")

In [5]:
# transformer 임포트 전에 환경 설정

# transformers 라이브러리가 온라인 모드로 동작하도록 설정
## 모델 다운로드 설정
os.environ["TRANSFORMERS_OFFLINE"] = "0"
# KLUE/BERT 모델 사용하기 위해 Hugging Face 모델 접근할 수 있도록 설정
os.environ["HF_HUB_OFFLINE"] = "0"

In [6]:
try:
    from transformers import AutoTokenizer, AutoModel, logging as transformers_logging

    transformers_logging.set_verbosity_error()  # 경고 메시지 최소화
    print("✅ Transformers 라이브러리 로드 완료")
except ImportError as e:
    print(f"❌ Transformers 라이브러리 임포트 실패: {e}")
    print("pip install transformers를 실행해주세요.")
    raise

✅ Transformers 라이브러리 로드 완료


In [7]:
## torch에 없는 함수 추가
torch.get_default_device = _get_default_device

## 데이터 전처리

In [8]:
# 한국어 불용어 리스트
KOREAN_STOPWORDS = [
    "이",
    "그",
    "저",
    "것",
    "및",
    "에",
    "를",
    "은",
    "는",
    "이런",
    "저런",
    "그런",
    "한",
    "이르",
    "또한",
    "있",
    "하",
    "에서",
    "으로",
    "으로써",
    "로써",
    "로서",
    "로",
    "와",
    "과",
    "이고",
    "이며",
    "이다",
    "있다",
    "하다",
    "되다",
    "이",
    "가",
    "을",
    "를",
    "에게",
    "의",
    "뿐",
    "다",
    "적",
    "데",
    "때",
    "나",
    "도",
    "만",
    "께",
    "에게서",
]

In [9]:
# 한국어 텍스트 전처리
def preprocess_korean_text(text):
    # 만약에 텍스트가 없으면 그냥 빈칸 반환
    if pd.isna(text) or text is None or len(str(text).strip()) == 0:
        return ""

    text = str(text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)  # 반복 문자 정규화
    text = re.sub(r"[^가-힣a-zA-Z0-9\s\.,!?]", " ", text)  # 특수문자 제거
    text = re.sub(r"\s+", " ", text).strip()  # 공백 정리

    return text


def remove_stopwords(text, stopwords=KOREAN_STOPWORDS):
    """주어진 텍스트에서 불용어 제거"""
    # 아무것도 없으면 그냥 반환
    if pd.isna(text) or text is None or len(str(text).strip()) == 0:
        return ""

    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]

    return " ".join(filtered_words)

## 감정 분류기 모델

In [10]:
class EmotionClassifier(torch.nn.Module):
    """감정 분류기 모델 - BERT 기반의 감정 분류를 위한 신경망 모델"""

    def __init__(self, bert_model, num_classes, dropout_rate=0.3):
        """
        모델 초기화

        Args:
            bert_model: 사전 훈련된 BERT 모델 (예: BertModel)
            num_classes: 분류할 감정 클래스의 개수
            dropout_rate: 드롭아웃 비율 (기본값: 0.3)
        """
        super(EmotionClassifier, self).__init__()

        # BERT 모델을 백본으로 사용
        self.bert = bert_model
        # BERT의 히든 사이즈 (일반적으로 768)
        self.hidden_size = self.bert.config.hidden_size

        # 과적합 방지를 위한 드롭아웃 레이어
        self.dropout1 = torch.nn.Dropout(dropout_rate)

        # 어텐션 메커니즘: 각 토큰의 중요도를 계산
        self.attention = torch.nn.Sequential(
            # 히든 사이즈를 256차원으로 축소
            torch.nn.Linear(self.hidden_size, 256),
            # Tanh 활성화 함수로 비선형성 추가
            torch.nn.Tanh(),
            # 256차원을 1차원으로 축소하여 어텐션 스코어 생성
            torch.nn.Linear(256, 1),
            # Softmax로 어텐션 가중치를 정규화 (합이 1이 되도록)
            torch.nn.Softmax(dim=1),
        )

        # 최종 분류를 위한 분류기
        self.classifier = torch.nn.Sequential(
            # 히든 사이즈를 256차원으로 축소
            torch.nn.Linear(self.hidden_size, 256),
            # ReLU 활성화 함수
            torch.nn.ReLU(),
            # 배치 정규화로 학습 안정성 향상
            torch.nn.BatchNorm1d(256),
            # 드롭아웃으로 과적합 방지
            torch.nn.Dropout(dropout_rate),
            # 최종 감정 클래스 개수만큼 출력
            torch.nn.Linear(256, num_classes),
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        순전파 과정

        Args:
            input_ids: 토크나이징된 입력 텍스트의 ID
            attention_mask: 패딩 토큰을 무시하기 위한 마스크
            token_type_ids: 문장 구분을 위한 토큰 타입 ID

        Returns:
            logits: 각 감정 클래스에 대한 점수
        """
        # BERT 모델에 입력을 통과시켜 임베딩 생성
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,  # 딕셔너리 형태로 결과 반환
        )

        # 모든 토큰의 히든 상태 (배치 크기, 시퀀스 길이, 히든 크기)
        sequence_output = outputs.last_hidden_state
        # [CLS] 토큰의 풀링된 표현 (배치 크기, 히든 크기)
        pooled_output = outputs.pooler_output

        # 어텐션 가중치 계산: 각 토큰의 중요도 결정
        attention_weights = self.attention(sequence_output)

        # 가중합을 통해 컨텍스트 벡터 생성
        # attention_weights와 sequence_output을 곱하고 시퀀스 차원을 따라 합산
        context_vector = torch.sum(attention_weights * sequence_output, dim=1)

        # 어텐션 기반 컨텍스트 벡터와 BERT의 풀링 출력을 결합
        # 두 표현을 더해서 더 풍부한 표현 생성
        final_output = context_vector + pooled_output

        # 드롭아웃 적용하여 과적합 방지
        final_output = self.dropout1(final_output)

        # 분류기를 통과시켜 최종 로짓 계산
        logits = self.classifier(final_output)

        return logits

## 계층적 감정 분류기

In [11]:
class HierarchicalEmotionClassifier:
    def __init__(self, model_dir, confidence_threshold=0.6, emotion_threshold=0.3):

        self.model_dir = model_dir  # 모델 경로 설정
        self.device = _get_default_device()  # 디바이스 설정
        self.max_len = 128  # 최대 길이 설정

        # 디버깅용
        print(f"모델 디렉토리: {self.model_dir}")
        print(f"디바이스: {self.device}")

        # 임계값 설정
        self.confidence_threshold = confidence_threshold  # 신뢰도 임계값
        self.emotion_threshold = emotion_threshold  # 감정 탐지 임계값

        print(f"신뢰도 임계값: {self.confidence_threshold}")
        print(f"감정 탐지 임계값: {self.emotion_threshold}")

        # 변수 초기화
        self.tokenizer = None
        self.bert_model = None
        self.level1_model = None
        self.level2_model = None
        self.level3_model = None
        self.level1_labels = None
        self.level2_labels = None
        self.level3_labels = None

        # 모델 로드
        self._load_models()

    def _check_files(self):
        print("1단계 : 파일 확인")
        # 필수 파일들 정의
        required_files = [
            "level1_best_model.pt",
            "level2_best_model.pt",
            "level3_best_model.pt",
            "level1_label_encoder.pkl",
            "level2_label_encoder.pkl",
            "level3_label_encoder.pkl",
        ]

        # 없는 파일 확인 리스트
        missing_files = []

        for file in required_files:
            filepath = os.path.join(self.model_dir, file)
            if not os.path.exists(filepath):
                missing_files.append(file)
            else:
                print(f"✅ {file} 파일이 존재합니다.")

        # 만약에 누락된 파일이 있으면 예외 발생
        if missing_files:
            raise FileExistsError(f"파일 누락 목록:{missing_files}")

    # 라벨 인코터 로드
    def _load_label_encoders(self):
        print("2단계: 라벨 인코더 로드")

        encoders = [
            ("level1_label_encoder.pkl", "level1_encoder"),
            ("level2_label_encoder.pkl", "level2_encoder"),
            ("level3_label_encoder.pkl", "level3_encoder"),
        ]

        for filename, attr_name in encoders:
            filepath = os.path.join(self.model_dir, filename)
            with open(filepath, "rb") as f:
                encoder = pickle.load(f)
                setattr(self, attr_name, encoder)
            print(f"{filename} 파일에서 {attr_name} 로드 완료")

    # 3단계 bert 모델 로드
    def _load_bert_model(self):

        try:
            print("3단계: BERT 모델 로드")
            # BERT 모델과 토크나이저 로드
            self.tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
            self.bert_model = AutoModel.from_pretrained("klue/bert-base")

            # 디바이스 이동
            self.bert_model = self.bert_model.to(self.device)
            print("BERT 모델과 토크나이저 로드 완료")
        except Exception as e:
            print(f"❌ BERT 모델 로드 실패: {e}")
            raise ImportError(
                "BERT 모델을 로드하는 데 실패했습니다. 'transformers' 라이브러리가 설치되어 있는지 확인하세요."
            )

    def _load_emotion_models(self):
        """감정 분류 모델들 로드"""
        print("😊 감정 분류 모델 로드 중...")

        models = [
            ("level1_best_model.pt", "level1_model", self.level1_encoder),
            ("level2_best_model.pt", "level2_model", self.level2_encoder),
            ("level3_best_model.pt", "level3_model", self.level3_encoder),
        ]

        for filename, attr_name, encoder in models:
            filepath = os.path.join(self.model_dir, filename)

            print(f"  🔄 {filename} 로드 중...")

            # 모델 초기화
            model = EmotionClassifier(
                copy.deepcopy(self.bert_model.to("cpu")), len(encoder.classes_)
            )

            # 가중치 로드
            try:
                state_dict = torch.load(filepath, map_location="cpu", weights_only=True)
            except TypeError:  # 구버전 PyTorch
                state_dict = torch.load(filepath, map_location="cpu")

            model.load_state_dict(state_dict)

            # 디바이스 이동 및 평가 모드
            model = model.to(self.device)
            model.eval()

            setattr(self, attr_name, model)
            print(f"  ✅ {filename} 로드 완료")

    def _load_models(self):
        """모든 모델 로드"""
        try:
            # 1단계 : 파일 존재 확인
            self._check_files()

            # 2단계 : 라벨 인코더 로드
            self._load_label_encoders()

            # 3단계
            self._load_bert_model()

            # 4단계
            self._load_emotion_models()

        except Exception as e:
            print(f"❌ 모델 로드 중 오류 발생: {e}")
            raise

    # 단일 모델 분류
    def _predict_single(self, model, text):
        # 전처리
        preprocessed = preprocess_korean_text(text)
        preprocessed = remove_stopwords(preprocessed)

        # 토큰화
        encoding = self.tokenizer.encode_plus(
            preprocessed,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        # 디바이스 이동
        input_ids = encoding["input_ids"].to(self.device)
        attention_mask = encoding["attention_mask"].to(self.device)
        token_type_ids = encoding["token_type_ids"].to(self.device)

        # 예측
        with torch.no_grad():  # 그래디어언트 계산 비활성화 : 메모리 절약 + 속도 향상
            outputs = model(input_ids, attention_mask, token_type_ids)  # 로짓 출력
            probs = torch.nn.functional.softmax(
                outputs, dim=1
            )  # 확률 변환, 각 클래스별 확률값
            _, preds = torch.max(outputs, dim=1)  # 가장 높은 확률을 가진 클래스 선택

        return preds.item(), probs[0]

    # 계층적 감정 분류
    def predict_hierarchical(self, text):
        """계층적 예측"""
        result = {
            "original_text": text,
            "preprocessed_text": remove_stopwords(preprocess_korean_text(text)),
            "levels": {},
        }

        # 1단계
        pred1, probs1 = self._predict_single(self.level1_model, text)
        label1 = self.level1_encoder.inverse_transform([pred1])[0]

        result["levels"]["level1"] = {
            "step": "1단계: 일반대화 vs 감정",
            "prediction": label1,
            "confidence": float(probs1[pred1]),
            "probabilities": {
                self.level1_encoder.classes_[i]: float(probs1[i])
                for i in range(len(self.level1_encoder.classes_))
            },
        }

        if label1 == "일반대화":
            result["final"] = {
                "prediction": "일반대화",
                "confidence": float(probs1[pred1]),
            }
            result["path"] = ["일반대화"]
            return result

        # 2단계
        pred2, probs2 = self._predict_single(self.level2_model, text)
        label2 = self.level2_encoder.inverse_transform([pred2])[0]

        result["levels"]["level2"] = {
            "step": "2단계: 기쁨 vs 기타감정",
            "prediction": label2,
            "confidence": float(probs2[pred2]),
            "probabilities": {
                self.level2_encoder.classes_[i]: float(probs2[i])
                for i in range(len(self.level2_encoder.classes_))
            },
        }

        if label2 == "기쁨":
            result["final"] = {"prediction": "기쁨", "confidence": float(probs2[pred2])}
            result["path"] = ["감정", "기쁨"]
            return result

        # 3단계
        pred3, probs3 = self._predict_single(self.level3_model, text)
        label3 = self.level3_encoder.inverse_transform([pred3])[0]

        result["levels"]["level3"] = {
            "step": "3단계: 세부 감정 분류",
            "prediction": label3,
            "confidence": float(probs3[pred3]),
            "probabilities": {
                self.level3_encoder.classes_[i]: float(probs3[i])
                for i in range(len(self.level3_encoder.classes_))
            },
        }

        result["final"] = {"prediction": label3, "confidence": float(probs3[pred3])}
        result["path"] = ["감정", "기타감정", label3]

        return result

    # 신뢰도 기반 복합 감정 분석
    def predict_with_confidence_analysis(self, text, threshold=0.55):
        result = self.predict_hierarchical(text)

        # 각 단계별 신뢰도 분석
        analysis = {
            "original_text": text,
            "complexity_level": "simple",  # simple, mixed, complex
            "primary_emotion": result["final"]["prediction"],
            "primary_confidence": result["final"]["confidence"],
            "mixed_emotions": [],
            "uncertainty_indicators": [],
        }

        # Level 2에서 기쁨과 기타감정 혼동 체크
        if "level2" in result["levels"]:
            level2_probs = result["levels"]["level2"]["probabilities"]
            joy_prob = level2_probs.get("기쁨", 0)
            other_prob = level2_probs.get("기타감정", 0)

            # 확률 차이가 작으면 혼합 감정으로 판단
            prob_diff = abs(joy_prob - other_prob)

            if prob_diff < 0.3:  # 30% 이하 차이면 혼합 감정
                analysis["complexity_level"] = "mixed"
                analysis["mixed_emotions"].append(
                    {"emotion": "기쁨", "confidence": joy_prob, "type": "positive"}
                )

                # Level 3에서 세부 감정도 추가
                if "level3" in result["levels"]:
                    level3_result = result["levels"]["level3"]
                    analysis["mixed_emotions"].append(
                        {
                            "emotion": level3_result["prediction"],
                            "confidence": level3_result["confidence"],
                            "type": "negative_or_neutral",
                        }
                    )
        if threshold is None:
            threshold = self.confidence_threshold

        # 전반적 신뢰도가 낮으면 복잡한 감정으로 분류
        if result["final"]["confidence"] < threshold:
            analysis["complexity_level"] = "complex"
            analysis["uncertainty_indicators"].append(
                f"낮은 신뢰도: {result['final']['confidence']:.3f}"
            )

        return analysis

    def predict_multi_emotion_threshold(self, text, threshold=None):
        """임계값 이상의 모든 감정 반환 (완성된 버전)"""
        # 임계값이 지정되지 않으면 클래스 기본값 사용
        if threshold is None:
            threshold = self.emotion_threshold

        result = {
            "original_text": text,
            "preprocessed_text": remove_stopwords(preprocess_korean_text(text)),
            "detected_emotions": [],
            "emotion_combination": None,
        }

        # 모든 단계에서 임계값 이상인 감정들 수집
        hierarchical_result = self.predict_hierarchical(text)

        all_emotions = []

        # Level 1 체크
        if "level1" in hierarchical_result["levels"]:
            for emotion, prob in hierarchical_result["levels"]["level1"][
                "probabilities"
            ].items():
                if prob >= threshold:
                    all_emotions.append(
                        {
                            "emotion": emotion,
                            "probability": prob,
                            "level": 1,
                            "category": "conversation_type",
                        }
                    )

        # Level 2 체크 (기쁨 vs 기타감정)
        if "level2" in hierarchical_result["levels"]:
            for emotion, prob in hierarchical_result["levels"]["level2"][
                "probabilities"
            ].items():
                if prob >= threshold:
                    all_emotions.append(
                        {
                            "emotion": emotion,
                            "probability": prob,
                            "level": 2,
                            "category": "emotion_polarity",
                        }
                    )

        # Level 3 체크 (세부 감정)
        if "level3" in hierarchical_result["levels"]:
            for emotion, prob in hierarchical_result["levels"]["level3"][
                "probabilities"
            ].items():
                if prob >= threshold:
                    all_emotions.append(
                        {
                            "emotion": emotion,
                            "probability": prob,
                            "level": 3,
                            "category": "specific_emotion",
                        }
                    )

        result["detected_emotions"] = sorted(
            all_emotions, key=lambda x: x["probability"], reverse=True
        )

        # 감정 조합 분석
        result["emotion_combination"] = self._analyze_emotion_combination(all_emotions)

        return result

    def _analyze_emotion_combination(self, emotions):
        """감정 조합 분석"""
        # Level 2에서 기쁨과 기타감정이 모두 임계값 이상인지 체크
        level2_emotions = [e for e in emotions if e["level"] == 2]
        level3_emotions = [e for e in emotions if e["level"] == 3]

        combination_type = "single"
        details = {}

        if len(level2_emotions) >= 2:  # 기쁨과 기타감정 둘 다 높은 확률
            joy_emotion = next(
                (e for e in level2_emotions if e["emotion"] == "기쁨"), None
            )
            other_emotion = next(
                (e for e in level2_emotions if e["emotion"] == "기타감정"), None
            )

            if joy_emotion and other_emotion:
                combination_type = "ambivalent"  # 양가감정
                details = {
                    "positive_aspect": {
                        "emotion": "기쁨",
                        "strength": joy_emotion["probability"],
                    },
                    "negative_aspect": {
                        "emotions": level3_emotions,
                        "primary": (
                            max(level3_emotions, key=lambda x: x["probability"])
                            if level3_emotions
                            else None
                        ),
                    },
                    "conflict_intensity": abs(
                        joy_emotion["probability"] - other_emotion["probability"]
                    ),
                }

        return {"type": combination_type, "details": details}

## 챗봇 생성

In [12]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
import yaml
import os


class EmotionChatbot:
    def __init__(
        self,
        model_dir,
        llm_provider="openai",
        confidence_threshold=0.6,
        emotion_threshold=0.3,
    ):
        # 감정 분류기 초기화
        self.classifier = HierarchicalEmotionClassifier(
            model_dir,
            confidence_threshold=confidence_threshold,
            emotion_threshold=emotion_threshold,
        )

        # 임계값을 챗봇에서도 저장 (필요시 사용)
        self.confidence_threshold = confidence_threshold
        self.emotion_threshold = emotion_threshold

        # 프롬프트 로드
        self.prompts = self._load_prompts()

        # llm 선정을 위해 여러 API 키를 환경 변수에서 불러옴
        try:
            # API 키 가져오기
            api_key = os.getenv("OPENAI_API_KEY")
            api_key2 = os.getenv("ANTHROPIC_API_KEY")

            if llm_provider == "openai":
                if not api_key:
                    raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

                self.llm = ChatOpenAI(
                    model="gpt-4o", temperature=0.4, max_tokens=1000, api_key=api_key
                )

            elif llm_provider == "claude":
                if not api_key2:
                    raise ValueError(
                        "ANTHROPIC_API_KEY 환경변수가 설정되지 않았습니다."
                    )

                self.llm = ChatAnthropic(
                    model="claude-opus-4-20250514",
                    temperature=0.4,
                    max_tokens=1000,
                    api_key=api_key2,
                )

            elif llm_provider == "gemini":
                try:
                    if not api_key3:
                        raise ValueError("GEMINI_API_KEY 환경변수가 설정되지 않았습니다.")
                    
                    print(f"Gemini API key 존재: {bool(api_key3)}")
                    print("ChatGoogleGenerativeAI 초기화 시도...")
                    
                    self.llm = ChatGoogleGenerativeAI(
                        model="gemini-1.5-pro-latest",
                        temperature=0.4,
                        api_key=api_key3,
                        

                    )
                    print("ChatGoogleGenerativeAI 초기화 완료")
        
                except Exception as gemini_error:
                    print(f"Gemini 초기화 중 오류: {gemini_error}")
                    raise

            # 메모리 초기화
            self.memory = ConversationSummaryBufferMemory(
                llm=self.llm,
                return_messages=True,
                max_token_limit=2000,
                memory_key="chat_history",
            )

            # 기본 프롬프트 설정
            self.system_prompt_text = self.prompts.get("basic_system")
            if not self.system_prompt_text:
                raise ValueError(
                    "basic_system 프롬프트가 로드되지 않았습니다. prompts/basic_system.yaml 파일을 확인해주세요."
                )

            self.chat_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_prompt_text),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("human", "{message}"),
                ]
            )

            self.chain = self.chat_prompt | self.llm
            print("✅ 챗봇 초기화 완료")

        except Exception as e:
            print(f"❌ 챗봇 초기화 중 오류 발생: {e}")
            raise

    def _load_prompts(self):
        """YAML 파일에서 프롬프트들을 로드"""
        prompts = {}

        # 프롬프트 파일들 리스트 (복합 감정 프롬프트만)
        prompt_files = {
            "basic_system": "prompts/basic_system.yaml",
            "ambivalent_empathy": "prompts/ambivalent_empathy.yaml",
            "mixed_validation": "prompts/mixed_validation.yaml",
            "exploratory_empathy": "prompts/exploratory_empathy.yaml",
            "direct_empathy": "prompts/direct_empathy.yaml",
            "general_conversation": "prompts/general_conversation.yaml",
        }

        for prompt_name, file_path in prompt_files.items():
            try:
                if os.path.exists(file_path):
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = yaml.safe_load(f)
                        # YAML 파일에서 'template' 키를 찾아서 사용
                        if isinstance(data, dict) and "template" in data:
                            prompts[prompt_name] = data["template"]
                        elif isinstance(data, str):
                            prompts[prompt_name] = data
                        else:
                            print(
                                f"⚠️ {file_path}에서 올바른 템플릿을 찾을 수 없습니다."
                            )
                else:
                    print(f"⚠️ {file_path} 파일이 존재하지 않습니다.")
            except Exception as e:
                print(f"❌ {file_path} 로드 중 오류: {e}")

        return prompts

    def classify_emotion(self, text):
        """기본 감정 분류"""
        return self.classifier.predict_hierarchical(text)

    def format_analysis(self, result):
        """기본 분석 결과 포맷팅"""
        output = f"""
📝 입력: {result['original_text']}
🔍 전처리: {result['preprocessed_text']}

🎯 최종 결과: {result['final']['prediction']} (신뢰도: {result['final']['confidence']:.4f})
📊 예측 경로: {' → '.join(result['path'])}

📈 단계별 분석:
"""

        for level, data in result["levels"].items():
            output += f"\n🔸 {data['step']}\n"
            output += f"   결과: {data['prediction']} ({data['confidence']:.4f})\n"
            output += f"   확률: "
            for emotion, prob in data["probabilities"].items():
                output += f"{emotion}({prob:.3f}) "
            output += "\n"

        return output

    def generate_response(self, text):
        """기본 챗봇 응답 생성 (메모리 추가)"""
        try:
            # 기본 감정 분류
            emotion_result = self.classify_emotion(text)

            # 감정 분석 포맷팅
            emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}, 경로: {' → '.join(emotion_result['path'])}"

            # 메모리에서 대화 기록 가져오기
            chat_history = self.memory.chat_memory.messages

            # 체인 실행
            response = self.chain.invoke(
                {
                    "message": text,
                    "emotion_info": emotion_info,
                    "chat_history": chat_history,
                }
            )

            # 응답 추출
            if hasattr(response, "content"):
                ai_response = response.content

                # 메모리에 대화저장
                self.memory.chat_memory.add_user_message(text)
                self.memory.chat_memory.add_ai_message(ai_response)

                return ai_response
            else:
                return "응답생성에 실패했습니다"

        except Exception as e:
            print(f"응답 생성 중 오류 발생: {e}")
            return "응답 생성 중 오류가 발생했습니다. 다시 시도해주세요."


    def generate_response_with_emotion_detection(self, text):
        """"감정 분류에 따라 응답 생성"""
        try:
            emotion_result=self.classify_emotion(text)
            
            chat_history=self.memory.chat_memory.messages

            #최종 예측 결과 확인
            finally_prediction=emotion_result['final']['prediction']

            if finally_prediction == "일반대화":
                #일반 대화 프롬프트 사용
                response=self.generate_general_conversation_response(text,emotion_result,chat_history)
            else:
                #감정이라면
                response=self.generate_emotion_response(text,emotion_result,chat_history)
            return response
        except Exception as e:
            print(f"감정 분류 및 응답 생성 중 오류 발생: {e}")
            return "죄송합니다. 응답 생성 중 문제가 발생했네요. 다시 말씀해 주시겠어요?"
    
    #일반대화 생성
    def generate_general_conversation_response(self,text, emotion_result, chat_history):
        """일반 대화 응답 생성"""
        try:
            emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}, 경로: {' → '.join(emotion_result['path'])}"

            #일반대화 프롬프트 구성
            general_chat_prompt= ChatPromptTemplate.from_messages(
                [
                    ("system",self.prompts['general_conversation']),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("message","{message}")
                ]
            )

            #체인 실행
            chain=general_chat_prompt | self.llm
            response=chain.invoke(
                {
                    "message":text,
                    "emotion_info": emotion_info,
                    "chat_history": chat_history,
                }
            )
            
            if hasattr(response, "content"):
                ai_response = response.content
                
                # 메모리에 대화 저장
                self.memory.chat_memory.add_user_message(text)
                self.memory.chat_memory.add_ai_message(ai_response)
                
                return ai_response
            else:
                return "응답 생성에 실패했습니다."
        
        except Exception as e:
            print(f"일반 대화 응답 생성 중 오류 발생: {e}")
            return "죄송합니다. 일반 대화 응답 생성 중 문제가 발생했네요. 다시 말씀해 주시겠어요?"
            




    def generate_empathetic_response(self, text):
        """복합 감정을 고려한 공감 응답 생성"""
        try:
            # 복합 감정 분석
            emotion_analysis = self.classifier.predict_with_confidence_analysis(text)
            multi_emotion_result = self.classifier.predict_multi_emotion_threshold(text)

            # 응답 전략 결정
            response_strategy = self._determine_response_strategy(
                emotion_analysis, multi_emotion_result
            )

            # 프롬프트에 복합 감정 정보 추가
            emotion_context = self._format_emotion_context(
                emotion_analysis, multi_emotion_result, response_strategy
            )

            # 메모리에서 대화 기록 가져오기
            chat_history = self.memory.chat_memory.messages

            # 개선된 프롬프트로 응답 생성
            response = self._generate_response_with_complex_emotion_context(
                text, emotion_context, chat_history
            )

            return response

        except Exception as e:
            print(f"공감 응답 생성 중 오류: {e}")
            return "죄송합니다. 응답 생성 중 문제가 발생했네요. 다시 말씀해 주시겠어요?"

    def _determine_response_strategy(self, emotion_analysis, multi_emotion_result):
        """응답 전략 결정"""
        if emotion_analysis["complexity_level"] == "mixed":
            if multi_emotion_result["emotion_combination"]["type"] == "ambivalent":
                return "ambivalent_empathy"  # 양가감정 공감
            else:
                return "mixed_validation"  # 혼합감정 인정
        elif emotion_analysis["complexity_level"] == "complex":
            return "exploratory_empathy"  # 탐색적 공감
        else:
            return "direct_empathy"  # 직접적 공감

    def _format_emotion_context(self, emotion_analysis, multi_emotion_result, strategy):
        """감정 맥락 포맷팅"""
        context = {
            "primary_emotion": emotion_analysis["primary_emotion"],
            "confidence": emotion_analysis["primary_confidence"],
            "complexity": emotion_analysis["complexity_level"],
            "response_strategy": strategy,
        }

        # 복합 감정 정보 추가
        if emotion_analysis["mixed_emotions"]:
            context["mixed_emotions"] = emotion_analysis["mixed_emotions"]

        # 양가감정 정보 추가
        if multi_emotion_result["emotion_combination"]["type"] == "ambivalent":
            context["ambivalent_details"] = multi_emotion_result["emotion_combination"][
                "details"
            ]

        return context

    def _generate_response_with_complex_emotion_context(
        self, text, emotion_context, chat_history
    ):
        """복합 감정 맥락을 고려한 응답 생성"""

        # 필수 프롬프트들이 모두 로드되었는지 확인
        required_prompts = [
            "ambivalent_empathy",
            "mixed_validation",
            "exploratory_empathy",
            "direct_empathy",
        ]
        missing_prompts = [
            p for p in required_prompts if p not in self.prompts or not self.prompts[p]
        ]

        if missing_prompts:
            raise ValueError(f"필수 프롬프트가 누락되었습니다: {missing_prompts}")

        # YAML에서 로드된 프롬프트 사용
        strategy_prompts = {}
        for strategy in required_prompts:
            strategy_prompts[strategy] = self.prompts[strategy]

        # 감정 정보 문자열 생성
        emotion_info_str = f"""
감정 분석 결과:
- 주요 감정: {emotion_context['primary_emotion']} (신뢰도: {emotion_context['confidence']:.2f})
- 복잡도: {emotion_context['complexity']}
- 응답 전략: {emotion_context['response_strategy']}
"""

        if "mixed_emotions" in emotion_context:
            emotion_info_str += "\n혼합 감정들:\n"
            for emotion in emotion_context["mixed_emotions"]:
                emotion_info_str += (
                    f"- {emotion['emotion']}: {emotion['confidence']:.2f}\n"
                )

        if "ambivalent_details" in emotion_context:
            details = emotion_context["ambivalent_details"]
            emotion_info_str += f"\n양가감정 세부사항:\n"
            emotion_info_str += f"- 긍정적 측면: {details['positive_aspect']['emotion']} ({details['positive_aspect']['strength']:.2f})\n"
            if details["negative_aspect"]["primary"]:
                emotion_info_str += f"- 부정적 측면: {details['negative_aspect']['primary']['emotion']} ({details['negative_aspect']['primary']['probability']:.2f})\n"

        # 전략에 맞는 시스템 프롬프트 선택
        system_prompt = strategy_prompts.get(
            emotion_context["response_strategy"], strategy_prompts["direct_empathy"]
        )

        # 프롬프트 구성
        chat_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{message}"),
            ]
        )

        # 체인 실행
        chain = chat_prompt | self.llm
        response = chain.invoke(
            {
                "message": text,
                "chat_history": chat_history,
                "emotion_info": emotion_info_str,
            }
        )

        if hasattr(response, "content"):
            ai_response = response.content
            # 메모리에 대화 저장
            self.memory.chat_memory.add_user_message(text)
            self.memory.chat_memory.add_ai_message(ai_response)
            return ai_response
        else:
            return "응답 생성에 실패했습니다."

    def reload_prompts(self):
        """프롬프트 재로드"""
        try:
            self.prompts = self._load_prompts()
            self.system_prompt_text = self.prompts.get("basic_system")
            if not self.system_prompt_text:
                raise ValueError(
                    "basic_system 프롬프트가 로드되지 않았습니다. prompts/basic_system.yaml 파일을 확인해주세요."
                )

            # 기본 체인 재생성
            self.chat_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_prompt_text),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("human", "{message}"),
                ]
            )
            self.chain = self.chat_prompt | self.llm

            print("✅ 프롬프트가 성공적으로 재로드되었습니다.")
        except Exception as e:
            print(f"❌ 프롬프트 재로드 중 오류: {e}")

    def process_message(self, text, include_ai_response=True):
        """기본 메시지 전체 처리"""
        # 감정 분석
        emotion_result = self.classify_emotion(text)
        emotion_analysis = self.format_analysis(emotion_result)

        result = {
            "emotion_analysis": emotion_analysis,
            "emotion_result": emotion_result,
        }

        # AI 응답
        if include_ai_response:
            ai_response = self.generate_response(text)
            result["ai_response"] = ai_response
            result["full_response"] = f"{emotion_analysis}\n💬 AI 상담:\n{ai_response}"
        else:
            result["full_response"] = emotion_analysis

        return result

    def process_complex_emotion_message(self, text):
        """복합 감정을 고려한 메시지 처리"""
        # 기본 감정 분석
        basic_result = self.classify_emotion(text)

        # 복합 감정 분석
        confidence_analysis = self.classifier.predict_with_confidence_analysis(text)
        multi_emotion_analysis = self.classifier.predict_multi_emotion_threshold(text)

        # 공감 응답 생성
        ai_response = self.generate_empathetic_response(text)

        # 결과 통합
        result = {
            "basic_emotion_analysis": self.format_analysis(basic_result),
            "confidence_analysis": confidence_analysis,
            "multi_emotion_analysis": multi_emotion_analysis,
            "ai_response": ai_response,
            "complexity_insight": self._generate_complexity_insight(
                confidence_analysis, multi_emotion_analysis
            ),
            "full_response": f"""
📊 감정 복잡성 분석:
{chr(10).join(self._generate_complexity_insight(confidence_analysis, multi_emotion_analysis))}

💬 AI 상담:
{ai_response}
""",
        }

        return result

    def _generate_complexity_insight(self, confidence_analysis, multi_emotion_analysis):
        """복잡성 인사이트 생성"""
        insights = []

        if confidence_analysis["complexity_level"] == "mixed":
            insights.append("🔀 혼합된 감정이 감지되었습니다.")

        if multi_emotion_analysis["emotion_combination"]["type"] == "ambivalent":
            insights.append("⚖️ 상반된 감정이 동시에 나타나는 양가감정 상태입니다.")

        if confidence_analysis["primary_confidence"] < 0.6:
            insights.append("🌊 감정이 복잡하고 미묘한 상태로 보입니다.")

        if len(multi_emotion_analysis["detected_emotions"]) > 3:
            insights.append("🎭 다양한 감정이 복합적으로 나타나고 있습니다.")

        return insights if insights else ["💡 비교적 명확한 감정 상태입니다."]

    # 메모리 관련 유틸리티 메서드들
    def get_conversation_summary(self):
        """현재 대화 요약 가져오기"""
        try:
            return self.memory.predict_new_summary(self.memory.chat_memory.messages, "")
        except Exception as e:
            print(f"대화 요약 생성 중 오류: {e}")
            return "요약을 생성할 수 없습니다."

    def get_chat_history(self):
        """전체 대화 기록 가져오기"""
        return self.memory.chat_memory.messages

    def clear_memory(self):
        """메모리 초기화"""
        self.memory.clear()
        print("💭 대화 기록이 초기화되었습니다.")

    def get_memory_status(self):
        """메모리 상태 확인"""
        try:
            messages = self.memory.chat_memory.messages
            token_count = self.memory.llm.get_num_tokens_from_messages(messages)

            return {
                "message_count": len(messages),
                "token_count": token_count,
                "max_token_limit": self.memory.max_token_limit,
                "is_summarizing": token_count > self.memory.max_token_limit,
            }
        except Exception as e:
            print(f"메모리 상태 확인 중 오류: {e}")
            return {
                "message_count": 0,
                "token_count": 0,
                "max_token_limit": self.memory.max_token_limit,
                "is_summarizing": False,
                "error": str(e),
            }

    def set_thresholds(self, confidence_threshold=None, emotion_threshold=None):
        """임계값 동적 변경"""
        if confidence_threshold is not None:
            self.confidence_threshold = confidence_threshold
            self.classifier.confidence_threshold = confidence_threshold
            print(f"신뢰도 임계값이 {confidence_threshold}로 변경되었습니다.")

        if emotion_threshold is not None:
            self.emotion_threshold = emotion_threshold
            self.classifier.emotion_threshold = emotion_threshold
            print(f"감정 탐지 임계값이 {emotion_threshold}로 변경되었습니다.")

    def get_thresholds(self):
        """현재 임계값 조회"""
        return {
            "confidence_threshold": self.confidence_threshold,
            "emotion_threshold": self.emotion_threshold,
        }
    def generate_emotion_response(self, text, emotion_result, chat_history):
        """감정 관련 프롬프트로 응답 생성 (기존 로직)"""
        try:
            # 감정 분석 포맷팅
            emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}, 경로: {' → '.join(emotion_result['path'])}"
            
            # 기본 감정 프롬프트 사용
            response = self.chain.invoke({
                "message": text,
                "emotion_info": emotion_info,
                "chat_history": chat_history,
            })
            
            if hasattr(response, "content"):
                ai_response = response.content
                
                # 메모리에 대화 저장
                self.memory.chat_memory.add_user_message(text)
                self.memory.chat_memory.add_ai_message(ai_response)
                
                return ai_response
            else:
                return "응답 생성에 실패했습니다"
                
        except Exception as e:
            print(f"감정 응답 생성 중 오류: {e}")
            return "죄송합니다. 감정 분석 응답 생성 중 문제가 발생했습니다."

    def process_message_with_detection(self, text, include_ai_response=True):
        """일반대화/감정 구분하여 메시지 처리"""
        # 감정 분석
        emotion_result = self.classify_emotion(text)
        emotion_analysis = self.format_analysis(emotion_result)
        
        result = {
            "emotion_analysis": emotion_analysis,
            "emotion_result": emotion_result,
            "conversation_type": emotion_result['final']['prediction']  # 일반대화 vs 감정
        }
        
        # AI 응답
        if include_ai_response:
            ai_response = self.generate_response_with_emotion_detection(text)
            result["ai_response"] = ai_response
            result["full_response"] = f"{emotion_analysis}\n\n💬 AI 응답:\n{ai_response}"
        else:
            result["full_response"] = emotion_analysis
        
        return result


# 챗봇 생성 헬퍼 함수
def create_chatbot(
    model_dir, llm_provider, confidence_threshold=0.6, emotion_threshold=0.3
):
    """챗봇 생성"""
    try:
        return EmotionChatbot(
            model_dir,
            llm_provider,
            confidence_threshold=confidence_threshold,
            emotion_threshold=emotion_threshold,
        )
    except Exception as e:
        print(f"❌ 챗봇 생성 실패: {e}")
        return None

## 테스트 해보기

In [14]:
# 모든 모델을 테스트해보세요
test_messages = [
    "오늘 너무 우울해서 아무것도 하기 싫어",
    "기쁘면서도 불안한 기분이야",
    "화가 나는데 동시에 슬퍼",
    "그냥 일상적인 대화를 하고 싶어"
]
model_dir = "/Users/hwangeunbi/chatnge_AI/chat/model"
for provider in ["claude", "openai", "gemini"]:
    chatbot = create_chatbot(model_dir, provider)
    print(result["ai_response"])
    # 각 메시지로 테스트하고 결과 비교

모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: mps
신뢰도 임계값: 0.6
감정 탐지 임계값: 0.3
1단계 : 파일 확인
✅ level1_best_model.pt 파일이 존재합니다.
✅ level2_best_model.pt 파일이 존재합니다.
✅ level3_best_model.pt 파일이 존재합니다.
✅ level1_label_encoder.pkl 파일이 존재합니다.
✅ level2_label_encoder.pkl 파일이 존재합니다.
✅ level3_label_encoder.pkl 파일이 존재합니다.
2단계: 라벨 인코더 로드
level1_label_encoder.pkl 파일에서 level1_encoder 로드 완료
level2_label_encoder.pkl 파일에서 level2_encoder 로드 완료
level3_label_encoder.pkl 파일에서 level3_encoder 로드 완료
3단계: BERT 모델 로드
BERT 모델과 토크나이저 로드 완료
😊 감정 분류 모델 로드 중...
  🔄 level1_best_model.pt 로드 중...
  ✅ level1_best_model.pt 로드 완료
  🔄 level2_best_model.pt 로드 중...
  ✅ level2_best_model.pt 로드 완료
  🔄 level3_best_model.pt 로드 중...
  ✅ level3_best_model.pt 로드 완료
✅ 챗봇 초기화 완료


NameError: name 'result' is not defined

In [21]:
# 사용 예시
if __name__ == "__main__":
    # 챗봇 생성
    model_dir = "/Users/hwangeunbi/chatnge_AI/chat/model"
    chatbot = create_chatbot(model_dir, llm_provider="gemini")

    if chatbot:
        print("✅ 챗봇이 성공적으로 생성되었습니다!")

        # 기본 사용
        result = chatbot.process_message_with_detection("오늘 좋은 일이 있었는데 뭔가 아쉬워")
        print(result["ai_response"])

        # 복합 감정 분석 사용
        complex_result = chatbot.process_complex_emotion_message("기쁘면서도 걱정돼")
        print(complex_result["full_response"])
    else:
        print("❌ 챗봇 생성에 실패했습니다.")

모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: mps
신뢰도 임계값: 0.6
감정 탐지 임계값: 0.3
1단계 : 파일 확인
✅ level1_best_model.pt 파일이 존재합니다.
✅ level2_best_model.pt 파일이 존재합니다.
✅ level3_best_model.pt 파일이 존재합니다.
✅ level1_label_encoder.pkl 파일이 존재합니다.
✅ level2_label_encoder.pkl 파일이 존재합니다.
✅ level3_label_encoder.pkl 파일이 존재합니다.
2단계: 라벨 인코더 로드
level1_label_encoder.pkl 파일에서 level1_encoder 로드 완료
level2_label_encoder.pkl 파일에서 level2_encoder 로드 완료
level3_label_encoder.pkl 파일에서 level3_encoder 로드 완료
3단계: BERT 모델 로드
BERT 모델과 토크나이저 로드 완료
😊 감정 분류 모델 로드 중...
  🔄 level1_best_model.pt 로드 중...
  ✅ level1_best_model.pt 로드 완료
  🔄 level2_best_model.pt 로드 중...
  ✅ level2_best_model.pt 로드 완료
  🔄 level3_best_model.pt 로드 중...
  ✅ level3_best_model.pt 로드 완료
Gemini API key 존재: True
ChatGoogleGenerativeAI 초기화 시도...
ChatGoogleGenerativeAI 초기화 완료
✅ 챗봇 초기화 완료
✅ 챗봇이 성공적으로 생성되었습니다!
좋은 일이 있었는데도 아쉬운 마음이 드는구나.  마음 한편이 텅 빈 것 같기도 하고, 뭔가 채워지지 않는 느낌일 수도 있겠네.  어떤 좋은 일이 있었는지, 그리고 그 일에서 어떤 부분이 아쉬웠는지 조금 더 이야기해 줄 수 있을까?  좋은 일 뒤에 아쉬움이 남는다는 게

In [85]:
# 사용 예시
if __name__ == "__main__":
    # 챗봇 생성
    model_dir = "/Users/hwangeunbi/chatnge_AI/chat/model"
    chatbot = create_chatbot(model_dir, llm_provider="openai")

    if chatbot:
        print("✅ 챗봇이 성공적으로 생성되었습니다!")

        # 기본 사용
        result = chatbot.process_message("오늘 좋은 일이 있었는데 뭔가 아쉬워")
        print(result["ai_response"])

        # 복합 감정 분석 사용
        complex_result = chatbot.process_complex_emotion_message("기쁘면서도 걱정돼")
        print(complex_result["full_response"])
    else:
        print("❌ 챗봇 생성에 실패했습니다.")

모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: mps
신뢰도 임계값: 0.6
감정 탐지 임계값: 0.3
1단계 : 파일 확인
✅ level1_best_model.pt 파일이 존재합니다.
✅ level2_best_model.pt 파일이 존재합니다.
✅ level3_best_model.pt 파일이 존재합니다.
✅ level1_label_encoder.pkl 파일이 존재합니다.
✅ level2_label_encoder.pkl 파일이 존재합니다.
✅ level3_label_encoder.pkl 파일이 존재합니다.
2단계: 라벨 인코더 로드
level1_label_encoder.pkl 파일에서 level1_encoder 로드 완료
level2_label_encoder.pkl 파일에서 level2_encoder 로드 완료
level3_label_encoder.pkl 파일에서 level3_encoder 로드 완료
3단계: BERT 모델 로드
BERT 모델과 토크나이저 로드 완료
😊 감정 분류 모델 로드 중...
  🔄 level1_best_model.pt 로드 중...
  ✅ level1_best_model.pt 로드 완료
  🔄 level2_best_model.pt 로드 중...
  ✅ level2_best_model.pt 로드 완료
  🔄 level3_best_model.pt 로드 중...
  ✅ level3_best_model.pt 로드 완료
✅ 챗봇 초기화 완료
✅ 챗봇이 성공적으로 생성되었습니다!
좋은 일이 있었는데도 아쉬움을 느끼고 있군요. 그런 감정이 들 때가 있죠. 그 좋은 일이 어떤 것이었는지, 그리고 무엇이 아쉬움을 남겼는지 조금 더 이야기해줄 수 있을까요? 그 감정의 배경을 함께 살펴보면 좋겠어요.

📊 감정 복잡성 분석:
🌊 감정이 복잡하고 미묘한 상태로 보입니다.

💬 AI 상담:
기쁜 일이 있었지만 동시에 걱정도 되는군요. 한편으로는 기쁨이 있지만, 다른 한편으로는 그 기쁨이 오래 가지 않을까 봐

In [86]:
# 사용 예시
if __name__ == "__main__":
    # 챗봇 생성
    model_dir = "/Users/hwangeunbi/chatnge_AI/chat/model"
    chatbot = create_chatbot(model_dir, llm_provider="claude")

    if chatbot:
        print("✅ 챗봇이 성공적으로 생성되었습니다!")

        # 기본 사용
        result = chatbot.process_message("오늘 좋은 일이 있었는데 뭔가 아쉬워")
        print(result["ai_response"])

        # 복합 감정 분석 사용
        complex_result = chatbot.process_complex_emotion_message("기쁘면서도 걱정돼")
        print(complex_result["full_response"])
    else:
        print("❌ 챗봇 생성에 실패했습니다.")

모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: mps
신뢰도 임계값: 0.6
감정 탐지 임계값: 0.3
1단계 : 파일 확인
✅ level1_best_model.pt 파일이 존재합니다.
✅ level2_best_model.pt 파일이 존재합니다.
✅ level3_best_model.pt 파일이 존재합니다.
✅ level1_label_encoder.pkl 파일이 존재합니다.
✅ level2_label_encoder.pkl 파일이 존재합니다.
✅ level3_label_encoder.pkl 파일이 존재합니다.
2단계: 라벨 인코더 로드
level1_label_encoder.pkl 파일에서 level1_encoder 로드 완료
level2_label_encoder.pkl 파일에서 level2_encoder 로드 완료
level3_label_encoder.pkl 파일에서 level3_encoder 로드 완료
3단계: BERT 모델 로드
BERT 모델과 토크나이저 로드 완료
😊 감정 분류 모델 로드 중...
  🔄 level1_best_model.pt 로드 중...
  ✅ level1_best_model.pt 로드 완료
  🔄 level2_best_model.pt 로드 중...
  ✅ level2_best_model.pt 로드 완료
  🔄 level3_best_model.pt 로드 중...
  ✅ level3_best_model.pt 로드 완료
✅ 챗봇 초기화 완료
✅ 챗봇이 성공적으로 생성되었습니다!
좋은 일이 있었는데도 마음 한편에 아쉬움이 남아있네요. 기쁨과 아쉬움이 함께 느껴지는 그런 복잡한 감정이 드시는 것 같아요.

좋은 일이 있었다고 하셨는데, 그 순간에도 뭔가 충분하지 않다고 느껴지셨나봐요. 혹은 그 좋은 일이 지나가버린 것에 대한 아쉬움일 수도 있겠네요.

어떤 부분이 특히 아쉽게 느껴지시나요? 그 좋은 일이 있을 때 어떤 기분이셨는지도 궁금해요.

이렇게 상반된 감정을 동시에 느끼는 건 자연스러운 일

In [15]:
import time
def test_single_model(model_dir, provider, test_messages):
    """단일 모델 테스트"""
    print(f"\n🤖 {provider.upper()} 테스트 중...")
    print("-" * 40)
    
    # 챗봇 생성
    try:
        chatbot = create_chatbot(model_dir, provider)
        if not chatbot:
            print(f"❌ {provider} 챗봇 생성 실패")
            return None
    except Exception as e:
        print(f"❌ {provider} 초기화 에러: {e}")
        return None
    
    results = []
    
    for i, message in enumerate(test_messages, 1):
        print(f"\n{i}. 입력: {message}")
        
        try:
            # 시간 측정 시작
            start_time = time.time()
            
            # 감정 분석
            emotion_result = chatbot.classify_emotion(message)
            
            # AI 응답 생성
            ai_response = chatbot.generate_response_with_emotion_detection(message)
            
            # 시간 측정 종료
            response_time = time.time() - start_time
            
            # 결과 출력
            print(f"   감정: {emotion_result['final']['prediction']}")
            print(f"   신뢰도: {emotion_result['final']['confidence']:.3f}")
            print(f"   응답시간: {response_time:.2f}초")
            print(f"   응답: {ai_response[:100]}...")
            
            results.append({
                'message': message,
                'emotion': emotion_result['final']['prediction'],
                'confidence': emotion_result['final']['confidence'],
                'response': ai_response,
                'time': response_time,
                'success': True
            })
            
        except Exception as e:
            print(f"   ❌ 에러: {e}")
            results.append({
                'message': message,
                'error': str(e),
                'time': 0,
                'success': False
            })
    
    return results


def compare_models(model_dir, providers, test_messages):
    """여러 모델 비교"""
    print("🚀 LLM 모델 비교 테스트 시작!")
    print("=" * 60)
    
    all_results = {}
    
    # 각 모델 테스트
    for provider in providers:
        results = test_single_model(model_dir, provider, test_messages)
        all_results[provider] = results
    
    # 결과 비교 출력
    print("\n📊 결과 비교")
    print("=" * 60)
    
    for provider, results in all_results.items():
        if results:
            success_count = sum(1 for r in results if r['success'])
            total_count = len(results)
            avg_time = sum(r['time'] for r in results if r['success']) / max(success_count, 1)
            avg_confidence = sum(r['confidence'] for r in results if r['success'] and 'confidence' in r) / max(success_count, 1)
            
            print(f"\n{provider.upper()}:")
            print(f"  ✅ 성공률: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
            print(f"  ⏱️ 평균 응답시간: {avg_time:.2f}초")
            print(f"  🎯 평균 신뢰도: {avg_confidence:.3f}")
    
    return all_results


def quick_test(model_dir):
    """빠른 테스트 (기본 케이스들)"""
    test_messages = [
        "오늘 너무 우울해",
        "기쁘면서도 불안해", 
        "화가 나는데 동시에 슬퍼",
        "안녕하세요",
        "날씨가 좋네요"
    ]
    
    providers = ["claude", "openai", "gemini"]
    
    return compare_models(model_dir, providers, test_messages)


def emotion_focused_test(model_dir):
    """감정 상담 특화 테스트"""
    emotion_test_cases = [
        "승진 소식을 들었는데 기쁘면서도 새로운 책임감 때문에 불안해",
        "친구와 싸웠는데 화가 나면서도 내가 잘못한 것 같아서 죄책감이 들어",
        "모든 게 무의미하게 느껴져",
        "아무도 나를 이해하지 못해",
        "혼자인 것 같아서 외로워",
        "오늘 정말 행복한 일이 있었어",
        "스트레스 받아서 미치겠어",
        "불안해서 잠이 안 와"
    ]
    
    providers = ["claude", "openai"]  # 감정 상담에 더 적합한 모델들
    
    return compare_models(model_dir, providers, emotion_test_cases)


def speed_test(model_dir):
    """응답 속도 테스트"""
    simple_messages = [
        "안녕",
        "좋아",
        "싫어",
        "도와줘",
        "고마워"
    ]
    
    print("⚡ 응답 속도 테스트")
    print("=" * 40)
    
    providers = ["claude", "openai", "gemini"]
    speed_results = {}
    
    for provider in providers:
        try:
            chatbot = create_chatbot(model_dir, provider)
            if not chatbot:
                continue
                
            times = []
            for message in simple_messages:
                start = time.time()
                chatbot.generate_response(message)
                times.append(time.time() - start)
            
            avg_time = sum(times) / len(times)
            speed_results[provider] = avg_time
            print(f"{provider.upper()}: {avg_time:.2f}초")
            
        except Exception as e:
            print(f"{provider.upper()}: 에러 - {e}")
    
    if speed_results:
        fastest = min(speed_results.items(), key=lambda x: x[1])
        print(f"\n🏆 가장 빠른 모델: {fastest[0].upper()} ({fastest[1]:.2f}초)")
    
    return speed_results


def detailed_comparison(model_dir, message):
    """특정 메시지에 대한 상세 비교"""
    print(f"🔍 상세 비교 - 입력: {message}")
    print("=" * 60)
    
    providers = ["claude", "openai", "gemini"]
    
    for provider in providers:
        print(f"\n🤖 {provider.upper()}")
        print("-" * 30)
        
        try:
            chatbot = create_chatbot(model_dir, provider)
            if not chatbot:
                print("❌ 챗봇 생성 실패")
                continue
            
            # 감정 분석
            start = time.time()
            emotion_result = chatbot.classify_emotion(message)
            emotion_time = time.time() - start
            
            # AI 응답
            start = time.time()
            ai_response = chatbot.generate_response_with_emotion_detection(message)
            response_time = time.time() - start
            
            print(f"감정 분석: {emotion_result['final']['prediction']} ({emotion_result['final']['confidence']:.3f})")
            print(f"감정 분석 시간: {emotion_time:.2f}초")
            print(f"응답 생성 시간: {response_time:.2f}초")
            print(f"총 시간: {emotion_time + response_time:.2f}초")
            print(f"응답:\n{ai_response}")
            
        except Exception as e:
            print(f"❌ 에러: {e}")


def recommendation_test(model_dir):
    """추천을 위한 종합 테스트"""
    print("🎯 종합 추천 테스트")
    print("=" * 50)
    
    # 다양한 테스트 실행
    print("1️⃣ 빠른 기본 테스트")
    basic_results = quick_test(model_dir)
    
    print("\n2️⃣ 감정 특화 테스트")  
    emotion_results = emotion_focused_test(model_dir)
    
    print("\n3️⃣ 속도 테스트")
    speed_results = speed_test(model_dir)
    
    # 종합 추천
    print("\n🏆 최종 추천")
    print("=" * 30)
    
    if speed_results:
        fastest = min(speed_results.items(), key=lambda x: x[1])
        print(f"⚡ 가장 빠른 모델: {fastest[0].upper()}")
    
    # 감정 상담용 추천 로직 (간단화)
    print("💭 감정 상담 챗봇 추천:")
    print("   1순위: Claude (감정적 공감 능력 우수)")
    print("   2순위: GPT-4 (균형잡힌 성능)")
    print("   3순위: Gemini (비용 효율적)")
    
    return {
        'basic': basic_results,
        'emotion': emotion_results, 
        'speed': speed_results
    }


# 실행 예시들
if __name__ == "__main__":
    MODEL_DIR = "/Users/hwangeunbi/chatnge_AI/chat/model" 
    
    # 1. 빠른 테스트 (2-3분)
    print("=== 빠른 테스트 ===")
    quick_test(MODEL_DIR)
    
    # 2. 특정 메시지 상세 비교
    print("\n=== 상세 비교 ===")
    detailed_comparison(MODEL_DIR, "오늘 기쁘면서도 불안해")
    
    # 3. 속도만 빠르게 체크
    print("\n=== 속도 테스트 ===")
    speed_test(MODEL_DIR)
    
    # 4. 종합 추천 테스트 (5-10분)
    # recommendation_test(MODEL_DIR)

=== 빠른 테스트 ===
🚀 LLM 모델 비교 테스트 시작!

🤖 CLAUDE 테스트 중...
----------------------------------------
모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: mps
신뢰도 임계값: 0.6
감정 탐지 임계값: 0.3
1단계 : 파일 확인
✅ level1_best_model.pt 파일이 존재합니다.
✅ level2_best_model.pt 파일이 존재합니다.
✅ level3_best_model.pt 파일이 존재합니다.
✅ level1_label_encoder.pkl 파일이 존재합니다.
✅ level2_label_encoder.pkl 파일이 존재합니다.
✅ level3_label_encoder.pkl 파일이 존재합니다.
2단계: 라벨 인코더 로드
level1_label_encoder.pkl 파일에서 level1_encoder 로드 완료
level2_label_encoder.pkl 파일에서 level2_encoder 로드 완료
level3_label_encoder.pkl 파일에서 level3_encoder 로드 완료
3단계: BERT 모델 로드
BERT 모델과 토크나이저 로드 완료
😊 감정 분류 모델 로드 중...
  🔄 level1_best_model.pt 로드 중...
  ✅ level1_best_model.pt 로드 완료
  🔄 level2_best_model.pt 로드 중...
  ✅ level2_best_model.pt 로드 완료
  🔄 level3_best_model.pt 로드 중...
  ✅ level3_best_model.pt 로드 완료
✅ 챗봇 초기화 완료

1. 입력: 오늘 너무 우울해


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   감정: 슬픔
   신뢰도: 0.574
   응답시간: 10.69초
   응답: 오늘 정말 우울한 마음이 드시는군요. 그런 무거운 감정을 안고 하루를 보내는 게 얼마나 힘드실지 느껴져요.

우울함이라는 감정이 마음 전체를 덮고 있는 것 같네요. 혹시 오늘 특별...

2. 입력: 기쁘면서도 불안해
   감정: 불안
   신뢰도: 0.932
   응답시간: 9.82초
   응답: 기쁨과 불안이 동시에 느껴지는군요. 이렇게 상반된 감정이 함께 있을 때는 마음이 더 복잡하고 혼란스러우실 것 같아요.

무언가 좋은 일이 생겼거나 기대되는 일이 있는데, 동시에 그...

3. 입력: 화가 나는데 동시에 슬퍼
   감정: 슬픔
   신뢰도: 0.924
   응답시간: 10.75초
   응답: 화와 슬픔이 동시에 밀려오는군요. 이 두 감정이 뒤엉켜 있을 때는 정말 마음이 무겁고 답답하실 것 같아요.

누군가에게 화가 나면서도, 그 상황이나 관계에 대해 슬픔을 느끼고 계신...

4. 입력: 안녕하세요
일반 대화 응답 생성 중 오류 발생: Unexpected message type: message. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.
   감정: 일반대화
   신뢰도: 1.000
   응답시간: 0.11초
   응답: 죄송합니다. 일반 대화 응답 생성 중 문제가 발생했네요. 다시 말씀해 주시겠어요?...

5. 입력: 날씨가 좋네요
   감정: 기쁨
   신뢰도: 1.000
   응답시간: 9.22초
   응답: 날씨가 좋다고 느끼시는군요. 맑은 날씨가 마음에도 작은 위안이 되고 있는 것 같아요.

방금 전까지 화와 슬픔이 뒤엉킨 복잡한 감정을 표현하셨는데, 이렇게 날씨 이야기로 화제를 바...

🤖 OPENAI 테스트 중...
----------------------------------------
모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: m